In [1]:
import mlflow
import os
import pandas as pd

from mlflow.tracking import MlflowClient
from mlflow.models import infer_signature
from mlflow.store.artifact.artifact_repository_registry import get_artifact_repository

from sklearn.datasets import fetch_california_housing
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [187]:
os.getenv("MLFLOW_TRACKING_URI", "No env")

'http://mlflow-service:5000'

In [188]:
mlflow.get_registry_uri()

'http://mlflow-service:5000'

# Управление экспериментами

In [193]:
exp_id = mlflow.create_experiment(name="test")

In [194]:
exp_id

'467883773888126917'

In [191]:
mlflow.delete_experiment(experiment_id=exp_id)

Не забудьте про rm -rf mlruns/trash/*

In [195]:
mlflow.search_experiments(
    filter_string="name = 'test'"
)

[<Experiment: artifact_location='s3://lizvladi-mlflow-artifacts/mlflow/467883773888126917', creation_time=1727798289091, experiment_id='467883773888126917', last_update_time=1727798289091, lifecycle_stage='active', name='test', tags={}>]

In [196]:
mlflow.search_experiments()

[<Experiment: artifact_location='s3://lizvladi-mlflow-artifacts/mlflow/467883773888126917', creation_time=1727798289091, experiment_id='467883773888126917', last_update_time=1727798289091, lifecycle_stage='active', name='test', tags={}>,
 <Experiment: artifact_location='s3://lizvladi-mlflow-artifacts/mlflow/281429030876195101', creation_time=1727787919129, experiment_id='281429030876195101', last_update_time=1727787919129, lifecycle_stage='active', name='elizveta_gavrilova', tags={}>,
 <Experiment: artifact_location='s3://lizvladi-mlflow-artifacts/parent_run_experiment', creation_time=1727785001137, experiment_id='637350945412675068', last_update_time=1727785001137, lifecycle_stage='active', name='parent_run_experiment', tags={}>,
 <Experiment: artifact_location='s3://lizvladi-mlflow-artifacts/mlflow/202052760323277175', creation_time=1725894166122, experiment_id='202052760323277175', last_update_time=1725894166122, lifecycle_stage='active', name='FINAL', tags={}>,
 <Experiment: artifa

In [197]:
mlflow.set_experiment(experiment_name="test")

with mlflow.start_run():
    # Обучим модель.
    model.fit(X_train, y_train)

2024/10/01 15:58:56 INFO mlflow.tracking._tracking_service.client: 🏃 View run rumbling-jay-171 at: http://mlflow-service:5000/#/experiments/467883773888126917/runs/5c8ab5a76751416aaa4752a13f05b38d.
2024/10/01 15:58:56 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow-service:5000/#/experiments/467883773888126917.


# Что можно отслеживать в рамках эксперимента

In [198]:
# Прочитаем данные.
housing = fetch_california_housing(as_frame=True)

In [199]:
X_train, X_test, y_train, y_test = train_test_split(housing['data'], housing['target'])
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5)

In [200]:
X_test.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
18940,5.0672,14.0,7.075665,1.018405,3196.0,3.267894,38.27,-122.06
5179,1.6389,44.0,4.438596,0.947368,1296.0,3.248120,33.95,-118.26
15683,5.7934,52.0,4.151565,1.069193,959.0,1.579901,37.80,-122.43
7488,3.4539,43.0,4.561069,0.977099,1251.0,4.774809,33.92,-118.22
5380,3.2222,24.0,3.836237,1.108014,491.0,1.710801,34.03,-118.40


## Первый запуск run

In [154]:
model = LinearRegression()

In [201]:
# Запустим свой первый run в рамках созданного выше эксперимента.
with mlflow.start_run(experiment_id=exp_id) as run:
    # Обучим модель.
    model.fit(X_train, y_train)

2024/10/01 16:00:17 INFO mlflow.tracking._tracking_service.client: 🏃 View run spiffy-lamb-687 at: http://mlflow-service:5000/#/experiments/467883773888126917/runs/9895cea9b4a24f2f90a8b5e31beb18b0.
2024/10/01 16:00:17 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow-service:5000/#/experiments/467883773888126917.


## Ручное логирование метрик

In [202]:
model = RandomForestRegressor()

In [203]:
params = {'n_estimators':50, 'min_samples_split':3}

In [204]:
# Запустим свой первый run в рамках созданного выше эксперимента.
with mlflow.start_run(experiment_id=exp_id) as run:

    # Обучим модель.
    model.fit(X_train, y_train)

    # Сделаем предсказание.
    prediction = model.predict(X_val)

    mlflow.log_metrics({'r2':r2_score(y_val, prediction),
                        'mse':mean_squared_error(y_val, prediction)})

    mlflow.log_params(params)

2024/10/01 16:01:29 INFO mlflow.tracking._tracking_service.client: 🏃 View run welcoming-ant-90 at: http://mlflow-service:5000/#/experiments/467883773888126917/runs/5d1e1f97996d4b60bf274675989ae58b.
2024/10/01 16:01:29 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow-service:5000/#/experiments/467883773888126917.


## Автоматическая оценка модели

In [205]:
# Запустим свой первый run в рамках созданного выше эксперимента.
with mlflow.start_run(experiment_id=exp_id) as run:
    # Обучим модель.
    model.fit(X_train, y_train)

    # Сделаем предсказание.
    prediction = model.predict(X_val)

    # Создадим валидационный датасет.
    eval_df = X_val.copy()
    eval_df["target"] = y_val
    eval_df["prediction"] = prediction

    # Сохраним результаты обучения с помощью MLFlow.
    mlflow.evaluate(
        data=eval_df,
        targets="target",
        predictions="prediction",
        model_type="regressor",
        evaluators=["default"],
    )

2024/10/01 16:04:05 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/10/01 16:04:05 WARNING mlflow.models.evaluation.default_evaluator: SHAP or matplotlib package is not installed, so model explainability insights will not be logged.
2024/10/01 16:04:05 INFO mlflow.tracking._tracking_service.client: 🏃 View run indecisive-rat-239 at: http://mlflow-service:5000/#/experiments/467883773888126917/runs/8208af17367444aaa2609c2472942d47.
2024/10/01 16:04:05 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow-service:5000/#/experiments/467883773888126917.


## Логирование модели в хранилище артефактов

In [160]:
model = LinearRegression()

In [206]:
# Запустим свой первый run в рамках созданного выше эксперимента.
with mlflow.start_run(experiment_id=exp_id) as run:
    # Обучим модель.
    model.fit(X_train, y_train)

    # Сделаем предсказание.
    prediction = model.predict(X_val)

    # Создадим валидационный датасет.
    eval_df = X_val.copy()
    eval_df["target"] = y_val
    #eval_df["prediction"] = prediction

    # Сохраним результаты обучения с помощью MLFlow.
    signature = infer_signature(housing['data'], prediction)
    model_info = mlflow.sklearn.log_model(model, 'logreg', signature=signature)
    mlflow.evaluate(
        model=model_info.model_uri,
        data=eval_df,
        targets="target",
        #predictions="prediction",
        model_type="regressor",
        evaluators=["default"],
    )

/opt/conda/lib/python3.11/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


2024/10/01 16:08:13 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2024/10/01 16:08:13 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/10/01 16:08:13 WARNING mlflow.models.evaluation.default_evaluator: SHAP or matplotlib package is not installed, so model explainability insights will not be logged.
2024/10/01 16:08:13 INFO mlflow.tracking._tracking_service.client: 🏃 View run powerful-asp-87 at: http://mlflow-service:5000/#/experiments/467883773888126917/runs/fbae7e11059741758382819e1f6b3767.
2024/10/01 16:08:13 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow-service:5000/#/experiments/467883773888126917.


In [162]:
os.environ["MLFLOW_ENABLE_SYSTEM_METRICS_LOGGING"] = "true"

In [163]:
# Запустим свой первый run в рамках созданного выше эксперимента.
with mlflow.start_run(experiment_id=exp_id) as run:
    # Обучим модель.
    model.fit(X_train, y_train)

    # Сделаем предсказание.
    prediction = model.predict(X_val)

    # Создадим валидационный датасет.
    eval_df = X_val.copy()
    eval_df["target"] = y_val
    #eval_df["prediction"] = prediction

    # Сохраним результаты обучения с помощью MLFlow.
    signature = infer_signature(housing['data'], prediction)
    model_info = mlflow.sklearn.log_model(model, 'logreg', signature=signature)
    mlflow.evaluate(
        model=model_info.model_uri,
        data=eval_df,
        targets="target",
        #predictions="prediction",
        model_type="regressor",
        evaluators=["default"],
    )

2024/10/01 12:40:10 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics because creating `GPUMonitor` failed with error: `pynvml` is not installed, to log GPU metrics please run `pip install pynvml` to install it..
2024/10/01 12:40:10 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.
/opt/conda/lib/python3.11/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing d

2024/10/01 12:40:15 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2024/10/01 12:40:15 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/10/01 12:40:15 WARNING mlflow.models.evaluation.default_evaluator: SHAP or matplotlib package is not installed, so model explainability insights will not be logged.
2024/10/01 12:40:15 INFO mlflow.tracking._tracking_service.client: 🏃 View run rare-sponge-111 at: http://mlflow-service:5000/#/experiments/593802373514970674/runs/e5d137e98bba46d79b1f689b1919fe1e.
2024/10/01 12:40:15 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow-service:5000/#/experiments/593802373514970674.
2024/10/01 12:40:15 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/10/01 12:40:15 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


In [164]:
os.environ["MLFLOW_ENABLE_SYSTEM_METRICS_LOGGING"] = "false"

In [165]:
# Удалим эксперимент из MLFlow.
mlflow.delete_experiment(experiment_id=exp_id)

In [166]:
# Удалим эксперимент из S3.
repository = get_artifact_repository(f's3://lizvladi-mlflow-artifacts/mlflow/{exp_id}')
repository.delete_artifacts()

## Работа с дочерними ранами

In [207]:
# Создать новый эксперимент
exp_name = "parent_run_experiment"
#experiment_id = mlflow.create_experiment(exp_name)
mlflow.set_experiment(exp_name)

<Experiment: artifact_location='s3://lizvladi-mlflow-artifacts/parent_run_experiment', creation_time=1727785001137, experiment_id='637350945412675068', last_update_time=1727785001137, lifecycle_stage='active', name='parent_run_experiment', tags={}>

In [208]:
models = dict(zip(["RandomForest", "LinearRegression", "HistGB"], 
                  [RandomForestRegressor(), LinearRegression(), HistGradientBoostingRegressor()]))

In [209]:
# Создадим parent run.
with mlflow.start_run(run_name="parent_run", experiment_id = experiment_id, description = "parent") as parent_run:
    for model_name in models.keys():
        # Запустим child run на каждую модель.
        with mlflow.start_run(run_name=model_name, experiment_id=experiment_id, nested=True) as child_run:
            model = models[model_name]
            
            # Обучим модель.
            model.fit(pd.DataFrame(X_train), y_train)
        
            # Сделаем предсказание.
            prediction = model.predict(X_val)
        
            # Создадим валидационный датасет.
            eval_df = X_val.copy()
            eval_df["target"] = y_val
        
            # Сохраним результаты обучения с помощью MLFlow.
            signature = infer_signature(X_test, prediction)
            model_info = mlflow.sklearn.log_model(model, "logreg", signature=signature)
            mlflow.evaluate(
                model=model_info.model_uri,
                data=eval_df,
                targets="target",
                model_type="regressor",
                evaluators=["default"],
            )

/opt/conda/lib/python3.11/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


2024/10/01 16:11:58 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2024/10/01 16:11:58 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/10/01 16:11:58 WARNING mlflow.models.evaluation.default_evaluator: SHAP or matplotlib package is not installed, so model explainability insights will not be logged.
2024/10/01 16:11:58 INFO mlflow.tracking._tracking_service.client: 🏃 View run RandomForest at: http://mlflow-service:5000/#/experiments/281429030876195101/runs/7757a8ec7dc2441db304db707dc69259.
2024/10/01 16:11:58 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow-service:5000/#/experiments/281429030876195101.
/opt/conda/lib/python3.11/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid u

2024/10/01 16:12:03 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2024/10/01 16:12:03 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/10/01 16:12:03 WARNING mlflow.models.evaluation.default_evaluator: SHAP or matplotlib package is not installed, so model explainability insights will not be logged.
2024/10/01 16:12:03 INFO mlflow.tracking._tracking_service.client: 🏃 View run LinearRegression at: http://mlflow-service:5000/#/experiments/281429030876195101/runs/1350974917be49f68e85cef99d148a34.
2024/10/01 16:12:03 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow-service:5000/#/experiments/281429030876195101.
/opt/conda/lib/python3.11/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avo

2024/10/01 16:12:13 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2024/10/01 16:12:13 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/10/01 16:12:13 WARNING mlflow.models.evaluation.default_evaluator: SHAP or matplotlib package is not installed, so model explainability insights will not be logged.
2024/10/01 16:12:13 INFO mlflow.tracking._tracking_service.client: 🏃 View run HistGB at: http://mlflow-service:5000/#/experiments/281429030876195101/runs/d36a6639963241d68d8a6c9f50711655.
2024/10/01 16:12:13 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow-service:5000/#/experiments/281429030876195101.
2024/10/01 16:12:13 INFO mlflow.tracking._tracking_service.client: 🏃 View run parent_run at: http://mlflow-service:5000/#/experiments/281429030876195101/runs/b6244cf9e2a8440a96ba1ea4591e4119.
2024/10/01 16:12:13 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflo

# Работа с реджистри

In [210]:
# Запустим свой первый run в рамках созданного выше эксперимента.
with mlflow.start_run() as run:
    # Обучим модель.
    model.fit(pd.DataFrame(X_train), y_train)

    # Сделаем предсказание.
    prediction = model.predict(X_val)

    # Создадим валидационный датасет.
    eval_df = X_val.copy()
    eval_df["target"] = y_val

    # Сохраним результаты обучения с помощью MLFlow.
    signature = infer_signature(X_test, prediction)
    model_info = mlflow.sklearn.log_model(model, 'linreg', signature=signature)
    mlflow.evaluate(
        model=model_info.model_uri,
        data=eval_df,
        targets="target",
        model_type="regressor",
        evaluators=["default"],
    )

/opt/conda/lib/python3.11/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


2024/10/01 16:17:01 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2024/10/01 16:17:01 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/10/01 16:17:01 WARNING mlflow.models.evaluation.default_evaluator: SHAP or matplotlib package is not installed, so model explainability insights will not be logged.
2024/10/01 16:17:01 INFO mlflow.tracking._tracking_service.client: 🏃 View run valuable-donkey-150 at: http://mlflow-service:5000/#/experiments/637350945412675068/runs/dbb5f886c2d647e59d964a5be27362c5.
2024/10/01 16:17:01 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow-service:5000/#/experiments/637350945412675068.


In [212]:
model_info.model_uri

'runs:/dbb5f886c2d647e59d964a5be27362c5/logreg'

Регистрация модели и версии способ 1

In [213]:
mlflow.register_model(model_uri=model_info.model_uri, name='linreg-model')

Successfully registered model 'linreg-model'.
2024/10/01 16:17:35 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: linreg-model, version 1
Created version '1' of model 'linreg-model'.


<ModelVersion: aliases=[], creation_timestamp=1727799455595, current_stage='None', description='', last_updated_timestamp=1727799455595, name='linreg-model', run_id='dbb5f886c2d647e59d964a5be27362c5', run_link='', source='s3://lizvladi-mlflow-artifacts/parent_run_experiment/dbb5f886c2d647e59d964a5be27362c5/artifacts/logreg', status='READY', status_message='', tags={}, user_id='', version='1'>

Регистрация версии способ 2

In [214]:
client = MlflowClient()

In [216]:
client.create_model_version(name='linreg-model', source=model_info.model_uri, run_id=run.info.run_id)

2024/10/01 16:19:44 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: linreg-model, version 2


<ModelVersion: aliases=[], creation_timestamp=1727799584334, current_stage='None', description='', last_updated_timestamp=1727799584334, name='linreg-model', run_id='dbb5f886c2d647e59d964a5be27362c5', run_link='', source='runs:/dbb5f886c2d647e59d964a5be27362c5/logreg', status='READY', status_message='', tags={}, user_id='', version='2'>

Регистрация версии способ 3

In [217]:
# Создадим parent run.
with mlflow.start_run(run_name="parent_run", experiment_id = experiment_id, description = "parent") as parent_run:
    for model_name in models.keys():
        # Запустим child run на каждую модель.
        with mlflow.start_run(run_name=model_name, experiment_id=experiment_id, nested=True) as child_run:
            model = models[model_name]
            
            # Обучим модель.
            model.fit(pd.DataFrame(X_train), y_train)
        
            # Сделаем предсказание.
            prediction = model.predict(X_val)

            # Создадим валидационный датасет.
            eval_df = X_val.copy()
            eval_df["target"] = y_val
        
            # Сохраним результаты обучения с помощью MLFlow.
            signature = infer_signature(X_test, prediction)
            model_info = mlflow.sklearn.log_model(model, "linreg", signature=signature, 
                                                  registered_model_name=f"sk-learn-{model_name}-reg-model")
            mlflow.evaluate(
                model=model_info.model_uri,
                data=eval_df,
                targets="target",
                model_type="regressor",
                evaluators=["default"],
            )

/opt/conda/lib/python3.11/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'sk-learn-RandomForest-reg-model'.
2024/10/01 16:20:50 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: sk-learn-RandomForest-reg-model, version 1
Created version '1' of model 'sk-learn-RandomForest-reg-model'.


2024/10/01 16:20:56 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2024/10/01 16:20:56 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/10/01 16:20:56 WARNING mlflow.models.evaluation.default_evaluator: SHAP or matplotlib package is not installed, so model explainability insights will not be logged.
2024/10/01 16:20:56 INFO mlflow.tracking._tracking_service.client: 🏃 View run RandomForest at: http://mlflow-service:5000/#/experiments/281429030876195101/runs/9532f97ad3334314aedd97517b00f4cb.
2024/10/01 16:20:56 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow-service:5000/#/experiments/281429030876195101.
/opt/conda/lib/python3.11/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid u

2024/10/01 16:21:01 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2024/10/01 16:21:01 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/10/01 16:21:01 WARNING mlflow.models.evaluation.default_evaluator: SHAP or matplotlib package is not installed, so model explainability insights will not be logged.
2024/10/01 16:21:01 INFO mlflow.tracking._tracking_service.client: 🏃 View run LinearRegression at: http://mlflow-service:5000/#/experiments/281429030876195101/runs/f082e1f0bc1340b692b497d9b06ed9db.
2024/10/01 16:21:01 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow-service:5000/#/experiments/281429030876195101.
/opt/conda/lib/python3.11/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avo

2024/10/01 16:21:09 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2024/10/01 16:21:09 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/10/01 16:21:09 WARNING mlflow.models.evaluation.default_evaluator: SHAP or matplotlib package is not installed, so model explainability insights will not be logged.
2024/10/01 16:21:09 INFO mlflow.tracking._tracking_service.client: 🏃 View run HistGB at: http://mlflow-service:5000/#/experiments/281429030876195101/runs/3322b4506cf54da88f325a248799cc9f.
2024/10/01 16:21:09 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow-service:5000/#/experiments/281429030876195101.
2024/10/01 16:21:09 INFO mlflow.tracking._tracking_service.client: 🏃 View run parent_run at: http://mlflow-service:5000/#/experiments/281429030876195101/runs/b7f35d1ab419468fb6c2549922069677.
2024/10/01 16:21:09 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflo

In [220]:
model = mlflow.sklearn.load_model(model_uri='s3://lizvladi-mlflow-artifacts/mlflow/281429030876195101/3322b4506cf54da88f325a248799cc9f/artifacts/linreg')

In [222]:
model.predict(X_test)

array([1.8427382 , 0.94766565, 4.7691683 , ..., 3.62601169, 2.77058856,
       2.32143739])